In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df = pd.read_csv("/kaggle/input/icc-test-cricket-runs/ICC Test Batting Figures.csv")

df[:10]

In [ ]:
df.shape

In [ ]:
df.info()

In [ ]:
df.columns

In [ ]:
object_to_int_list = ['Inn', 'NO', 'Runs', 'Avg', '100', '50','0']
for i in object_to_int_list:
    df[i] = df[i].str.replace("-", "0")
    df[i] = pd.to_numeric(df[i])

In [ ]:
df.info()

In [ ]:
# Create new column for countries
df["Country"] = df["Player"].str.extract('.*\((.*)\).*', expand=True)
df[:10]

In [ ]:
df["Country"].unique()

In [ ]:
# Remove "/ICC" and "ICC/" from the "Country" column
df["Country"] = df["Country"].str.replace("/ICC", "")
df["Country"] = df["Country"].str.replace("ICC/", "")
df["Country"].unique()


In [ ]:
# Number of players that have represented 2 countries
two_countries = df[df["Country"].str.contains("/")]
two_countries

In [ ]:
# Change "Country" of players that represented 2 countries to "Two countries"    
two_country_player = two_countries["Player"].tolist()

two_country_player_country = two_countries["Country"].unique().tolist()

for i in two_country_player:
    player = df[df["Player"] == i]
    for c in two_country_player_country:
        df["Country"] = df["Country"].str.replace(c, "Two countries")

In [ ]:
df["Country"].unique()

In [ ]:
df["Country"].value_counts()

In [ ]:
# Show percentage of players per country
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.style

matplotlib.style.use("bmh")

ax = df["Country"].value_counts().plot(figsize = (15,10), kind = "pie")


In [ ]:
# TOTALLY STOLE THIS FROM GEEKS FOR GEEKS 
# https://www.geeksforgeeks.org/plot-a-pie-chart-in-python-using-matplotlib/

# Creating explode data
explode = (0.1, 0.2, 0.1, 0.1, 0.1, 0.2, 0.1, 0.1, 0.2, 0.1, 0.2, 0.1, 0.2)

# Creating color parameters
colors = ( "orange", "cyan", "purple" , "blue", "yellow", "red", "green", 
          "white", "brown", "grey", "indigo", "beige", "silver" )

# Wedge properties
wp = { 'linewidth' : 2, 'edgecolor' : "green" }

# Creating autocpt arguments
def func(pct, allvalues):
    absolute = int(pct / 100.*np.sum(allvalues))
    return "{:.1f}%\n({:d} players)".format(pct, absolute)

# Creating plot
fig, ax = plt.subplots(figsize =(30, 15))
wedges, texts, autotexts = ax.pie(df["Country"].value_counts(), 
                                  autopct = lambda pct: func(pct, df["Country"].value_counts()),
                                  explode = explode, 
                                  labels = df["Country"].unique(),
                                  shadow = True,
                                  colors = colors,
                                  startangle = 90,
                                  wedgeprops = wp,
                                  textprops = dict(color ="black"))
  
# Adding legend
ax.legend(wedges, df["Country"].unique(),
          title ="Counries",
          loc ="center left",
          bbox_to_anchor =(1, 0, 0.5, 1))
  
plt.setp(autotexts, size = 8, weight ="bold")
# ax.set_title("Player country split")
  
# show plot
plt.show()

In [ ]:
# Runs scored, 100s, 50s and ducks per country
country_grouped = df.groupby("Country").sum()
runs_per_country = country_grouped.sort_values("Runs", ascending = False)
runs_per_country = runs_per_country[["Runs", "100", "50", "0"]]
runs_per_country

In [ ]:
# Country with the best coversion rate
runs_per_country["Conversion rate"] = round(runs_per_country["100"] / 
                                            (runs_per_country["100"] + runs_per_country["50"])
                                            * 100, 2)
runs_per_country.sort_values("Conversion rate", ascending = False)